# Projeto 2 - Ciência dos Dados

Nome: Beatriz Lourenço

Nome: Gabriel Yamashita

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
import re 
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@Reme72277305***

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [5]:
#Produto escolhido:
produto = "McDonalds"

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break
# shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [7]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [16]:
data = pd.read_excel("McDonalds.xlsx",sheet_name = "Treinamento")
colunas = ["Treinamento", "relevancia"]
data = data.loc[:,colunas]

def cleanup(text):
    punctuation = "[!\-.:?;,'#-]"
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

data = cleanup(data.lower())
# data_treino = data['Treinamento'].apply(cleanup)

# data_treino = pd.Series(data_treino.split())
# data_treino
data

In [27]:
rel = " ".join(data[data["relevancia"] == 1].Treinamento)
irrel = " ".join(data[data["relevancia"] == 0].Treinamento)

rel = cleanup(rel)
irrel = cleanup(irrel)

rel = pd.Series(rel.split())
irrel = pd.Series(irrel.split())

In [28]:
rel_value = rel.value_counts()
rel_value

@mcdonalds_br       33
e                   28
de                  27
o                   22
que                 19
no                  19
mcdonalds           15
do                  15
eu                  12
@ifood              11
não                 11
a                   11
nunca               10
é                   10
um                  10
uma                  7
tem                  7
da                   6
//t                  6
https                6
comer                5
só                   5
pedido               5
pelo                 5
pedi                 4
mais                 4
se                   4
esse                 4
@samianainternet     4
mista                4
                    ..
devo                 1
febre                1
😂❤️                  1
onde                 1
preparo              1
aí                   1
meus                 1
super                1
embu                 1
quem                 1
papelão              1
pensar               1
impossível 

In [29]:
irrel_value = irrel.value_counts()
irrel_value

mcdonalds         139
de                115
@mcdonalds_br     101
que                82
e                  72
o                  72
do                 71
a                  62
eu                 61
rt                 56
//t                56
não                56
é                  56
https              56
no                 54
um                 52
com                40
em                 33
uma                29
da                 28
pra                28
por                26
na                 24
q                  20
minha              20
você               18
2                  18
mc                 17
se                 17
me                 17
                 ... 
dangerous           1
facebook)           1
trabalha            1
invenção            1
desnecessários      1
🙏🏻❤️🌹               1
falar               1
now                 1
@smith_hays         1
larson              1
ccpc                1
facissu             1
@gomesataner        1
co/em72skimpe       1
mães      

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**